# Image classification

This tutorial shows how to classify images of flowers using a `tf.keras.Sequential` model and load data using `tf.keras.utils.image_dataset_from_directory`. It demonstrates the following concepts:


* Efficiently loading a dataset off disk.
* Identifying overfitting and applying techniques to mitigate it, including data augmentation and dropout.
This tutorial follows a basic machine learning workflow:

1. Examine and understand data
2. Build an input pipeline
3. Build the model
4. Train the model
5. Test the model
6. Improve the model and repeat the process


## Setup

Import TensorFlow and other necessary libraries:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
import pathlib
import warnings

## Use this treminal commands to inastall all the libraries at once 

In [ ]:
# pip install numpy
# pip install opencv
# pip install PIL
# pip install tensorflow
# pip install matplotlib
# pip install pathlib

## Download and explore the dataset

This tutorial uses a dataset of 3,670 photos of flowers. The dataset contains five sub-directories, one per class:

```
flower_photo/
  daisy/
  dandelion/
  roses/
  sunflowers/
  tulips/
```

In [ ]:
import pathlib

dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos.tar', origin=dataset_url, extract=True)
data_dir = pathlib.Path(data_dir).with_suffix('')

After downloading, you should now have a copy of the dataset available. There are 3,670 total images:

# loading the dataset Locally 

In [ ]:
local_dataset_path = "PATH\image recognition model\\flower_photos"
data_dir = pathlib.Path(local_dataset_path).with_suffix('').resolve()

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

# Visaulizing a sample of the dataset

Loading images of roses and plots the first two images using matplotlib, The images are opened using the PIL library.

In [ ]:
roses = list(data_dir.glob('roses/*'))
# Plot the first two images of roses
plt.figure(figsize=(10, 5))
for i in range(2):
    plt.subplot(1, 2, i + 1)
    image = PIL.Image.open(str(roses[i]))
    plt.imshow(image)
    plt.title('Rose Image {}'.format(i + 1))
    plt.axis('off')
plt.show()

loading images of tulips and plots the first two images using matplotlib, The images are opened using the PIL library.

In [ ]:
tulips = list(data_dir.glob('tulips/*'))
# Plot the first two images of tulips
plt.figure(figsize=(10, 5))    #initializes a new figure with a size of 10 inches by 5 inches.
for i in range(2):    # iterates twice, for i values 0 and 1, to plot two images.
    plt.subplot(1, 2, i + 1)    # sets up a subplot layout of 1 row and 2 columns, and selects the i-th subplot.
    image = PIL.Image.open(str(tulips[i]))    # opens the i-th tulip image file.
    plt.imshow(image)
    plt.title('Tulip Image {}'.format(i + 1))    # This line sets the title for each subplot to 'Tulip Image 1' and 'Tulip Image 2'.
    plt.axis('off')    # removes the axis labels and ticks from the plots.
plt.show()

## Load data using a Keras utility

Next, load these images off disk using the helpful `tf.keras.utils.image_dataset_from_directory` utility. This will take you from a directory of images on disk to a `tf.data.Dataset` in just a couple lines of code. If you like, you can also write your own data loading code from scratch by visiting the [Load and preprocess images](../load_data/images.ipynb) tutorial.

### Create a dataset

Define the initial parameters for the image processing:
- batch_size : sets the number of images per batch. 
- img_height & img_width : define the dimensions to which each image will be resized.

In [ ]:
batch_size = 64
img_height = 180
img_width = 180

It's good practice to use a validation split when developing your model. Use 80% of the images for training and 20% for validation.


- `data_dir` : The directory where the data is located.
- `validation_split=0.8` : Specifies that 80% of the data should be used for training.
- `subset="training"` : Indicates this is the training subset of the data.
- `seed=123` : A random seed for shuffling and splitting the data, ensuring reproducibility.
- `image_size=(img_height, img_width)` : Resizes all images to the specified height and width.
- `batch_size=batch_size` : The number of images to include in each batch (64 images per batch).
- `verbose=0` : Suppresses verbose output during dataset loading.

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.8,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    verbose=0)

- `data_dir` : The directory where the data is located.
- `validation_split=0.2` : Specifies that 20% of the data should be used for training.
- `subset="validation"` : Indicates this is the training subset of the data.
- `seed=123` : A random seed for shuffling and splitting the data, ensuring reproducibility.
- `image_size=(img_height, img_width)` : Resizes all images to the specified height and width.
- `batch_size=batch_size` : The number of images to include in each batch (64 images per batch).
- `verbose=0` : Suppresses verbose output during dataset loading.

In [ ]:
validation_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

You can find the class names in the `class_names` attribute on these datasets. These correspond to the directory names in alphabetical order.

In [ ]:
class_names = train_ds.class_names
print(class_names)

## Visualize the data

Here are the first nine images from the training dataset:

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

You will pass these datasets to the Keras `Model.fit` method for training later in this tutorial. If you like, you can also manually iterate over the dataset and retrieve batches of images:

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

The `image_batch` is a tensor of the shape `(32, 180, 180, 3)`. This is a batch of 64 images of shape `180x180x3` (the last dimension refers to color channels RGB). The `label_batch` is a tensor of the shape `(32,)`, these are corresponding labels to the 32 images.

You can call `.numpy()` on the `image_batch` and `labels_batch` tensors to convert them to a `numpy.ndarray`.


## Configure the dataset for performance

Make sure to use buffered prefetching, so you can yield data from disk without having I/O become blocking. These are two important methods you should use when loading data:

- `Dataset.cache` keeps the images in memory after they're loaded off disk during the first epoch. This will ensure the dataset does not become a bottleneck while training your model. If your dataset is too large to fit into memory, you can also use this method to create a performant on-disk cache.
- `Dataset.prefetch` overlaps data preprocessing and model execution while training.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = validation_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Standardize the data

The RGB channel values are in the `[0, 255]` range. This is not ideal for a neural network; in general you should seek to make your input values small.

Here, you will standardize values to be in the `[0, 1]` range by using `tf.keras.layers.Rescaling`:

In [ ]:
normalization_layer = layers.Rescaling(1./255)

There are two ways to use this layer. You can apply it to the dataset by calling `Dataset.map`:

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]

The pixel values are now in `[0,1]`.

In [ ]:
print("min value for pixel ", np.min(first_image))
print("max value for pixel ", np.max(first_image))

## processing the images
To prepare it for input into the model.

The `preprocess_image` function simplifies the preprocessing pipeline for image data, ensuring that images are properly formatted and ready for consumption by machine learning models.

- `img_array = tf.keras.utils.img_to_array(img)`: converts the image to a NumPy array.
- `img_array = tf.expand_dims(img_array, 0)` : extra dimension to the array to create a batch. This is necessary because many TensorFlow operations expect the batch dimension.

In [ ]:
def preprocess_image(image_path):
    img = tf.keras.utils.load_img(image_path, target_size=(img_height, img_width))
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    return img_array

Or, you can include the layer inside your model definition, which can simplify deployment. Use the second approach here.

Note: You previously resized images using the `image_size` argument of `tf.keras.utils.image_dataset_from_directory`. If you want to include the resizing logic in your model as well, you can use the `tf.keras.layers.Resizing` layer.

# Overfitting

the training accuracy is increasing linearly over time, whereas validation accuracy stalls around 60% in the training process. Also, the difference in accuracy between training and validation accuracy is noticeable  a sign of [overfitting](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit).

When there are a small number of training examples, the model sometimes learns from noises or unwanted details from training examples to an extent that it negatively impacts the performance of the model on new examples. This phenomenon is known as overfitting. It means that the model will have a difficult time generalizing on a new dataset.

There are multiple ways to fight overfitting in the training process. In this tutorial, you'll use *data augmentation* and add *dropout* to your model.

## Data augmentation 
Overfitting generally occurs when there are a small number of training examples. `Data augmentation` takes the approach of generating additional training data from your existing examples by augmenting them using random transformations that yield believable-looking images. This helps expose the model to more aspects of the data and generalize better.

- `layers.RandomFlip("horizontal", input_shape=(img_height, img_width, 3))` : This layer applies random horizontal flips to the input images.
`"horizontal"` specifies the direction of the flip.
`input_shape=(img_height, img_width, 3)` defines the shape of the input images.( 3 color channels for RGB images).
- `layers.RandomRotation(0.1)` : This layer applies random rotations to the input images.
`0.1` specifies the maximum angle of rotation in radians.
- `layers.RandomZoom(0.1)` : This layer applies random zooms to the input images.
`0.1` specifies the range of zoom.


In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal",
                          input_shape=(img_height,
                                       img_width,
                                       3)),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
    ]
)

Visualize a few augmented examples by applying data augmentation to the same image several times:

In [ ]:
# Visualize a few augmented examples by applying data augmentation to the same image several times
plt.figure(figsize=(8, 8))
for images, _ in train_ds.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.title('Augmentation sample')
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")

## Creating a check point to callback later

Preventing Loss of Progress: If training is interrupted, you can resume from the last saved checkpoint without starting from scratch.
Model Performance: By saving the weights at each epoch or specified interval, you can keep track of the model's performance at different stages of training and potentially roll back to a better-performing set of weights if needed.

`ModelCheckpoint` : This is a callback provided by Keras that saves the model or its weights at certain intervals during training.

- `filepath=checkpoint_path` : Specifies the path where the checkpoint will be saved, as defined earlier.

- `save_weights_only=True` : This parameter ensures that only the model's weights are saved, not the entire model architecture. This is useful if you want to save storage space and only need the weights to reload the model later.

- `verbose=1` : This parameter enables verbose output, which means that information about the saving process will be printed to the console during training.


In [ ]:

checkpoint_path = "PATH\\training_checkpoint/cp.weights.h5"

checkpoint_callback = ModelCheckpoint(filepath=checkpoint_path,
                                    save_weights_only=True,
                                    verbose=1)

# Training Up The model

#### Create the model

The Keras [Sequential](https://www.tensorflow.org/guide/keras/sequential_model) model consists of three convolution blocks (`tf.keras.layers.Conv2D`) with a max pooling layer (`tf.keras.layers.MaxPooling2D`) in each of them. There's a fully-connected layer (`tf.keras.layers.Dense`) with 128 units on top of it that is activated by a ReLU activation function (`'relu'`). This model has not been tuned for high accuracy; the goal of this tutorial is to show a standard approach.

In [ ]:
num_classes = len(class_names)

model = Sequential([
    data_augmentation,
        layers.Rescaling(1. / 255, input_shape=(img_height, img_width, 3)),
        layers.Conv2D(16, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(32, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Dropout(0.2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, name="outputs")
    ])

`num_classes` : The total number of classes in the dataset, derived from the length of class_names.

- `model = Sequential([...])`: linear stack of layers for building the neural network model.<br>
list of layers is passed to the Sequential() constructor, specifying the layers of the model in sequential order.

`Rescaling`
- `(1. / 255)` : Scales the pixel values from the range [0, 255] to [0, 1].<br>
- `input_shape=(img_height, img_width, 3)` : Specifies the shape of the input images (height, width, 3 color channels).


1st convolutional layer<br>
`Conv2D(16, 3)` : A convolutional layer with 16 filters, each of size 3x3.<br>
`padding='same'` : Ensures the output has the same width and height as the input.<br>
`activation='relu'` : Uses the ReLU activation function.

1st Max Pooling layer<br>
`MaxPooling2D()` : A max-pooling layer that reduces the spatial dimensions (height and width) by taking the maximum value in each 2x2 window.

2nd convolutional layer<br>
`Conv2D(32, 3)` : A convolutional layer with 32 filters, each of size 3x3.

2nd Max Pooling layer<br>
`MaxPooling2D()`

3rd convolutional layer<br>
`Conv2D(64, 3)` : A convolutional layer with 64 filters, each of size 3x3.

3rd Max Pooling layer<br>
`MaxPooling2D()`

`Dropout(0.2)` : A dropout layer that randomly sets 20% of the input units to 0 at each update during training time to prevent overfitting.

First Dense (Fully Connected) Layer:<br>
`Dense(128)` : A fully connected layer with 128 neurons.

Output Layer :
- `Dense(num_classes)` : A fully connected layer with a neuron for each class in the dataset (output layer).
- `name="outputs"` : The name of the layer, typically used for identification purposes.

## Compile the model

For this tutorial, choose the `tf.keras.optimizers.Adam` optimizer and `tf.keras.losses.SparseCategoricalCrossentropy` loss function. To view training and validation accuracy for each training epoch, pass the `metrics` argument to `Model.compile`.

In [ ]:
model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

`optimizer='adam'` :the Adam optimizer, which is an adaptive learning rate optimization algorithm. Adam combines the best properties of the AdaGrad and RMSProp algorithms to provide an optimization algorithm that can handle sparse gradients on noisy problems.<br>


`loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)x` :<br>
Specifies the loss function to be used during training.
- `SparseCategoricalCrossentropy` : This loss function is used for multi-class classification problems where the target variable is in integer form (sparse labels).
- `from_logits=True` : Indicates that the model's output values are raw logits. This parameter ensures that the loss function applies the necessary transformation to the logits.


`metrics=['accuracy']` : Specifies the metric to be evaluated by the model during training and testing.
- `accuracy` : This metric calculates how often predictions match the labels. It is commonly used for classification problems to give a quick sense of model performance.

## Model summary

View all the layers of the network using the Keras `Model.summary` method:

In [ ]:
model.summary()

## Train the model


Train the model for 50 epochs with the Keras `Model.fit` method:


In [ ]:
iterations = 50
history = model.fit(
      train_ds,
      validation_data=validation_ds,
      epochs=iterations,
      callbacks=[checkpoint_callback]
)

## Save&Load The Model:
Saving the entire trained model to a file and attempts to load the weights from a specified checkpoint, handling potential errors if the checkpoint file is not found. It ensures the model's state can be preserved and reloaded for continued training or inference.<br>
This allows you to reload the model later without needing to rebuild its architecture and ensures that the model uses the weights saved during training.


In [ ]:
model.save('my_model.keras')
model.load_weights(checkpoint_path)
try:
    model.load_weights(checkpoint_path)
    print("Weights loaded successfully!")
except FileNotFoundError:
    print("Checkpoint file not found. Please verify the file path.")

`model.save('my_model.keras')` : Saves the entire model, including its architecture, weights, and training configuration, to a file named `my_model.keras`.<br>
`model.load_weights(checkpoint_path)` : Loads the model's weights from the specified checkpoint file located at checkpoint_path.




## Visualize training results

Create plots of the loss and accuracy on the training and validation sets:

In [ ]:
# Visualize training results accuracy and loss
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

iterations_range = range(iterations)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(iterations_range, acc, label='Training Accuracy')
plt.plot(iterations_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(iterations_range, loss, label='Training Loss')
plt.plot(iterations_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Predict on new data

Use your model to classify an image that wasn't included in the training or validation sets.

Note: Data augmentation and dropout layers are inactive at inference time.

In [ ]:
image_path = "https://storage.googleapis.com/download.tensorflow.org/example_images/592px-Red_sunflower.jpg"
image_path = tf.keras.utils.get_file('Red_sunflower', origin=image_path)

model = load_model(model_path)
preprocessed_image = preprocess_image(image_path)
predictions = model.predict(preprocessed_image)
predicted_class_index = np.argmax(predictions)
class_names = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']
predicted_class = class_names[predicted_class_index]
confidence = np.max(predictions) * 100

print(f"The image most likely belongs to {predicted_class} with a confidence of {confidence:.2f}%.")
